In [117]:
import pandas
pandas.set_option('display.max_rows',1000)
pandas.set_option('display.max_columns',10000)
import numpy

In [57]:
m2016 = pandas.read_csv('music2016.csv')
m2016 = m2016.drop(m2016.columns[0], axis=1)
m2016

,artist,title,rating,url,chords
0,Justin Bieber,LOVE YOURSELF,4527.0,https://tabs.ultimate-guitar.com/j/justin_bieb...,C G/B Am Dm C G/B C G/B Am Dm C G/B C G/B Am D...
1,Justin Bieber,SORRY,737.0,https://tabs.ultimate-guitar.com/j/justin_bieb...,F Am G F Am G F Am G F Am G F Am G F Am G F Am...
2,Drake Featuring WizKid & Kyla,ONE DANCE,113.0,https://tabs.ultimate-guitar.com/d/drake/one_d...,Bbm C# Ebm Ebm Bbm C# Ebm Ebm Bbm C# Ebm Ebm B...
3,Rihanna Featuring Drake,WORK,86.0,https://tabs.ultimate-guitar.com/r/rihanna/wor...,Am7 G/B Cadd9 D Am7 G/B Cadd9 D Am7 G/B Cadd9 ...
4,twenty one pilots,STRESSED OUT,887.0,https://tabs.ultimate-guitar.com/t/twenty_one_...,Am C Dm E Em F G F Dm Am Am F Dm Am F Dm Am F ...
5,Adele,HELLO,1032.0,https://tabs.ultimate-guitar.com/a/adele/hello...,Em G D C Bm Em G D C Em G D C Em G D C Em G D ...
6,The Chainsmokers Featuring Daya,DON'T LET ME DOWN,284.0,https://tabs.ultimate-guitar.com/t/the_chainsm...,D A E F#m D A E F#m D A E F#m D A E F#m D A E ...
7,Justin Timberlake,CAN'T STOP THE FEELING!,156.0,https://tabs.ultimate-guitar.com/j/justin_timb...,C Am F Am C Am F Am C Am F Am C Am F Am C Am F...
8,The Chainsmokers Featuring Halsey,CLOSER,606.0,https://tabs.ultimate-guitar.com/t/the_chainsm...,Cadd9 D Em7 D Cadd9 D Em7 D Cadd9 D Em7 D Cadd...
9,Sia Featuring Sean Paul,CHEAP THRILLS,316.0,https://tabs.ultimate-guitar.com/s/sia/cheap_t...,F#m D A E F#m D A E F#m D A E F#m D A E F#m D ...


In [58]:
m2016['chord_set'] = [set(chords.split(' ')) for chords in m2016.chords]
cd_2016_set = set().union(*m2016.chord_set)

In [59]:
cd_2016_set

{'A',
 'A5',
 'Ab',
 'Ab/Bb',
 'Am',
 'Am/C',
 'Am/E',
 'Am/G',
 'Am7',
 'B',
 'B5',
 'B7',
 'Bb',
 'Bb/C',
 'Bbm',
 'Bm',
 'C',
 'C#',
 'C#m',
 'C/B',
 'C/E',
 'C/G',
 'C5',
 'Cadd9',
 'Cm',
 'Cmaj7',
 'D',
 'D/F#',
 'Db',
 'Dbsus2',
 'Dm',
 'Dm7',
 'Dmaj',
 'Dsus2',
 'Dsus4',
 'E',
 'E/G#',
 'E5',
 'Eb',
 'Ebm',
 'Em',
 'Em/C#',
 'Em/G',
 'Em7',
 'F',
 'F#',
 'F#/D',
 'F#11',
 'F#M',
 'F#m',
 'Fm',
 'Fm7',
 'G',
 'G#m',
 'G/Ab',
 'G/B',
 'G/E',
 'G7',
 'Gb',
 'Gm',
 'Gmaj7'}

In [80]:
chord_pair = list()
for chords in m2016.chords:
    cd_list = chords.split(' ')
    for index in range(0,len(cd_list)-1):
        temp_str = str(cd_list[index] + ' ' + cd_list[index+1])
        chord_pair.append(temp_str)
set(chord_pair)

{'A A',
 'A B',
 'A Bm',
 'A C',
 'A D',
 'A Dmaj',
 'A E',
 'A Em',
 'A F#',
 'A F#m',
 'A G',
 'A5 B5',
 'Ab Ab',
 'Ab Bb',
 'Ab Bbm',
 'Ab Db',
 'Ab Gb',
 'Ab/Bb Bb',
 'Ab/Bb C',
 'Am Am',
 'Am Am/E',
 'Am Am/G',
 'Am B',
 'Am Bb/C',
 'Am C',
 'Am C/E',
 'Am C/G',
 'Am D',
 'Am Dm',
 'Am Dm7',
 'Am E',
 'Am Em',
 'Am F',
 'Am G',
 'Am/C Dm',
 'Am/C Em',
 'Am/E Am/C',
 'Am/G Dm',
 'Am7 D',
 'Am7 F',
 'Am7 G/B',
 'B B',
 'B B7',
 'B C',
 'B C#',
 'B C#m',
 'B D',
 'B E',
 'B G',
 'B G#m',
 'B5 C5',
 'B7 C',
 'B7 Em',
 'B7 G',
 'Bb Ab',
 'Bb Cm',
 'Bb Eb',
 'Bb Fm7',
 'Bb/C C',
 'Bbm Ab',
 'Bbm Bbm',
 'Bbm C#',
 'Bm A',
 'Bm Am',
 'Bm Bm',
 'Bm C',
 'Bm D',
 'Bm E',
 'Bm Em',
 'Bm G',
 'C Ab/Bb',
 'C Am',
 'C Am7',
 'C B',
 'C B7',
 'C Bb/C',
 'C Bm',
 'C C',
 'C C/B',
 'C D',
 'C Dm',
 'C E',
 'C Em',
 'C Em7',
 'C F',
 'C F#/D',
 'C G',
 'C G/B',
 'C G7',
 'C# B',
 'C# Bbm',
 'C# C#',
 'C# Ebm',
 'C#m A',
 'C#m D',
 'C/B Am',
 'C/E Am',
 'C/E Dm',
 'C/E F',
 'C/G F',
 'C5 E5',
 'Cadd

In [110]:
cd0 = m2016.chords[0]
cd_pair = list()
cd0_list = cd0.split(' ')
for i in range(0, len(cd0_list)-1):
    temp_str = str(cd0_list[i] + ' ' + cd0_list[i+1])
    cd_pair.append(temp_str)
cd_pair_set = set(cd_pair)

In [112]:
cd_pair_set

{'Am Dm',
 'Am F',
 'C Am',
 'C C',
 'C G',
 'C G/B',
 'Dm C',
 'F C',
 'F G',
 'G Am',
 'G C',
 'G/B Am',
 'G/B C'}

In [116]:
cd0_list.count('Am')

28

In [128]:
def pmi(co_curr, a_curr, b_curr, doc_size):
    ratio = (co_curr*doc_size) / ((a_curr-co_curr) * (b_curr-co_curr))
    PMI = numpy.log2(ratio)
    return PMI

In [129]:
pmi(1159, 1938, 1311, 50000000)

18.900682578600115

In [ ]:
import mah